# DAT500 - FinalProject, Flight Delay Prediction

Group Members:

    - Brage Solheim
    - Yohannes D. Kassaye

## Introduction

So far we have used map reduce jobs to find null values in our data, find mean and mode of each column, and fill null values with mean or mode.

We will be predicting flight delays based on data that is available prior to the day of the flight. <br>
Such data could be weather, airport, airline, origin and destination, and other factors.

## Initlization

In [2]:
import findspark
findspark.init()
findspark.find()

'/usr/local/spark'

In [3]:
from datetime import datetime, date
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql import SparkSession
import pyspark.pandas as ps

In [4]:
ORIGINAL_DATA_PATH = "hdfs:///data/airline_delay_data.csv"
CLEANED_DATA_PATH = "hdfs:///data/airline_delay_data_cleaned.csv"

TEST_DATA_ORIGINAL_PATH = "/data/2015.sample.csv"
TEST_DATA_CLEANED_PATH = "/data/2015.sample.cleaned.csv"

In [5]:
import warnings
warnings.filterwarnings("ignore")

In [6]:
import pyspark
conf = pyspark.SparkConf().setAll([("spark.driver.maxResultSize", '4g'), ("spark.sql.execution.arrow.pyspark.enabled", 'true')])

In [7]:
spark = (SparkSession
            .builder
            .master("yarn") # for cluster
            # .master("local[*]") # for local testing
            .config(conf=conf)
            .appName("Flight-delay-prediction")
            .getOrCreate())

sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2022-04-21 08:45:11,650 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-04-21 08:45:18,896 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [8]:
print(sc.getConf().getAll())

[('spark.ui.proxyBase', '/proxy/application_1645016454320_0104'), ('spark.driver.port', '45379'), ('spark.sql.warehouse.dir', 'file:/home/ubuntu/DAT500-FinalProject/spark/spark-warehouse'), ('spark.executor.id', 'driver'), ('spark.app.id', 'application_1645016454320_0104'), ('spark.app.name', 'Flight-delay-prediction'), ('spark.driver.host', 'namenode'), ('spark.master', 'yarn'), ('spark.app.startTime', '1650530711442'), ('spark.executorEnv.PYTHONPATH', '/usr/local/spark/python/:<CPS>{{PWD}}/pyspark.zip<CPS>{{PWD}}/py4j-0.10.9.3-src.zip'), ('spark.rdd.compress', 'True'), ('spark.org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter.param.PROXY_URI_BASES', 'http://namenode:8088/proxy/application_1645016454320_0104'), ('spark.serializer.objectStreamReset', '100'), ('spark.submit.pyFiles', ''), ('spark.yarn.isPython', 'true'), ('spark.submit.deployMode', 'client'), ('spark.ui.filters', 'org.apache.hadoop.yarn.server.webproxy.amfilter.AmIpFilter'), ('spark.org.apache.hadoop.yarn.serve

In [9]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DateType, DoubleType

In [10]:
features = ["FL_DATE","OP_CARRIER","OP_CARRIER_FL_NUM","ORIGIN","DEST","CRS_DEP_TIME","DEP_TIME","DEP_DELAY","TAXI_OUT","WHEELS_OFF","WHEELS_ON","TAXI_IN","CRS_ARR_TIME","ARR_TIME","ARR_DELAY","CANCELLED","DIVERTED","CRS_ELAPSED_TIME","ACTUAL_ELAPSED_TIME","AIR_TIME","DISTANCE","CARRIER_DELAY","WEATHER_DELAY","NAS_DELAY","SECURITY_DELAY","LATE_AIRCRAFT_DELAY"]
nullable=False
schema = StructType([
    StructField("FL_DATE", DateType(), nullable), StructField("OP_CARRIER", StringType(), nullable), StructField("OP_CARRIER_FL_NUM", DoubleType(), nullable),
    StructField("ORIGIN", StringType(), nullable), StructField("DEST", StringType(), nullable), StructField("CRS_DEP_TIME", DoubleType(), nullable),
    StructField("DEP_TIME", DoubleType(), nullable), StructField("DEP_DELAY", DoubleType(), nullable), StructField("TAXI_OUT", DoubleType(), nullable),
    StructField("WHEELS_OFF", DoubleType(), nullable), StructField("WHEELS_ON", DoubleType(), nullable), StructField("TAXI_IN", DoubleType(), nullable),
    StructField("CRS_ARR_TIME", DoubleType(), nullable), StructField("ARR_TIME", DoubleType(), nullable), StructField("ARR_DELAY", DoubleType(), nullable),
    StructField("CANCELLED", DoubleType(), nullable), StructField("DIVERTED", DoubleType(), nullable), StructField("CRS_ELAPSED_TIME", DoubleType(), nullable),
    StructField("ACTUAL_ELAPSED_TIME", DoubleType(), nullable), StructField("AIR_TIME", DoubleType(), nullable), StructField("DISTANCE", DoubleType(), nullable),
    StructField("CARRIER_DELAY", DoubleType(), nullable), StructField("WEATHER_DELAY", DoubleType(), nullable), StructField("NAS_DELAY", DoubleType(), nullable),
    StructField("SECURITY_DELAY", DoubleType(), nullable), StructField("LATE_AIRCRAFT_DELAY", DoubleType(), nullable)])

In [11]:
df = spark.read.csv(CLEANED_DATA_PATH, header=True, schema=schema, inferSchema=False)
df.to_pandas_on_spark().head()

2022-04-21 08:47:07,650 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 08:47:07,699 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-21 08:47:07,740 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2016-09-25,AA,2622.0,EGE,DFW,800.0,749.0,-11.0,11.0,800.0,1049.0,15.0,1110.0,1104.0,-6.0,0.0,0.0,130.0,135.0,109.0,721.0,19.62,3.04,15.08,0.09,24.72
1,2016-09-25,AA,2623.0,DFW,TPA,1830.0,1915.0,45.0,20.0,1935.0,2243.0,3.0,2155.0,2246.0,51.0,0.0,0.0,145.0,151.0,128.0,929.0,0.00,0.00,6.00,0.00,45.00
2,2016-09-25,AA,2625.0,MCO,DFW,935.0,930.0,-5.0,12.0,942.0,1052.0,17.0,1124.0,1109.0,-15.0,0.0,0.0,169.0,159.0,130.0,985.0,19.62,3.04,15.08,0.09,24.72
3,2016-09-25,AA,2626.0,DFW,FLL,1429.0,1710.0,161.0,28.0,1738.0,2054.0,4.0,1814.0,2058.0,164.0,0.0,0.0,165.0,168.0,136.0,1119.0,15.00,0.00,3.00,0.00,146.00
4,2016-09-25,AA,2626.0,TUS,DFW,915.0,910.0,-5.0,10.0,920.0,1337.0,14.0,1328.0,1351.0,23.0,0.0,0.0,133.0,161.0,137.0,813.0,0.00,0.00,23.00,0.00,0.00


In [13]:
df = df.repartition(24)

In [14]:
from pyspark.sql import functions as F

In [15]:
header_row = df.filter(F.col("FL_DATE").isNull())
header_row.to_pandas_on_spark()

2022-04-19 14:38:23,106 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-19 14:38:23,144 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-19 14:39:29,736 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,None,OP_CARRIER,NaN,ORIGIN,DEST,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We see that there is one row with null values do to the header being in the middle of our data, and when we infer the schema, the header row is not the same type as the rest of the rows, and therefore is converted to null.

In [16]:
df = df.filter((F.col("FL_DATE").isNull() & F.col("OP_CARRIER").isNotNull() & F.col("ORIGIN").isNotNull() & F.col("DEST").isNotNull()) == False)

In [17]:
header_row = df.filter(F.col("FL_DATE").isNull())
print(header_row.count())

0


Next we will encode categorical columns, because we will be using them in our model.

In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

categorical_columns = [col[0] for col in df.dtypes if col[1] == "string"]
indexers = [StringIndexer(inputCol=col, outputCol=col+"_indexed", handleInvalid="keep").fit(df) for col in categorical_columns]

pipeline = Pipeline(stages=indexers)
df_indexed = pipeline.fit(df).transform(df)
df_indexed.show(5)

+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+
|   FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|
+----------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+------------------

We are also dealing with time series data, and therefore we need to encode the dates.

In [20]:
from pyspark.sql.functions import unix_timestamp

df_indexed = df_indexed.withColumn("FL_DATE", F.col("FL_DATE").cast("timestamp"))\
             .withColumn("FL_DATE_indexed", unix_timestamp(F.col("FL_DATE"), format="yyyy-MM-dd"))
df_indexed.show(5)

+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+------------------+--------------+------------+---------------+
|            FL_DATE|OP_CARRIER|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|WHEELS_OFF|WHEELS_ON|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|OP_CARRIER_indexed|ORIGIN_indexed|DEST_indexed|FL_DATE_indexed|
+-------------------+----------+-----------------+------+----+------------+--------+---------+--------+----------+---------+-------+------------+--------+---------+---------+--------+----------------+-------------------+--------+--------+------------

## Data Analysis

So far we have cleaned the data by removing and filling null values.
Now we will try to find the correlation between the our target, DEP_DELAY, and the other columns.

In [26]:
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

In [27]:
def GetCorrelationMatrix(df):
    inputcols = [col[0] for col in df_indexed.dtypes if col[1] == "double" or col[1] == "int" or col[1] == "bigint"]
    vector_col = "features"

    assembler = VectorAssembler(inputCols=inputcols, outputCol=vector_col, handleInvalid="skip")
    df_vector = assembler.transform(df).select(vector_col)
    matrix = Correlation.corr(df_vector, "features")

    matrix_df = spark.createDataFrame(matrix.collect()[0]["pearson({})".format(vector_col)].toArray().tolist(), inputcols).to_pandas_on_spark()
    matrix_df["feature"] = inputcols

    return matrix_df
    

In [28]:
corr_matrix = GetCorrelationMatrix(df_indexed)

In [30]:
DEP_DELAY_CORR = corr_matrix[["DEP_DELAY", "feature"]].sort_values(by="DEP_DELAY", ascending=False)
DEP_DELAY_CORR

2022-04-19 15:15:27,056 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-19 15:15:27,209 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,DEP_DELAY,feature
3,1.000000,DEP_DELAY
10,0.947662,ARR_DELAY
17,0.524859,CARRIER_DELAY
21,0.420768,LATE_AIRCRAFT_DELAY
18,0.215824,WEATHER_DELAY
2,0.147498,DEP_TIME
5,0.137768,WHEELS_OFF
19,0.121006,NAS_DELAY
1,0.105527,CRS_DEP_TIME
8,0.090892,CRS_ARR_TIME


We see that there are some features with negative correlations with respect to our target feature, so we will remove those.

High degree: If the coefficient value lies between ± 0.50 and ± 1, then it is said to be a strong correlation. Moderate degree: If the value lies between ± 0.30 and ± 0.49, then it is said to be a medium correlation. Low degree: When the value lies below + . 29, then it is said to be a small correlation.

In [31]:
DEP_DELAY_CORR.loc[DEP_DELAY_CORR["DEP_DELAY"] > 0.5]

2022-04-19 15:16:43,568 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
2022-04-19 15:16:43,712 WARN window.WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


,DEP_DELAY,feature
3,1.000000,DEP_DELAY
10,0.947662,ARR_DELAY
17,0.524859,CARRIER_DELAY


Need to figure out if using negative treshold for correlation is a good idea. It looks like the features affecting DEP_DELAY is the ones over

----

Further data analysis....

----

## What will we predict?

We will be predicting flight delays based on data that is available prior to the day of the flight. <br>
Such data could be weather, airport, airline, origin and destination, and other factors.

## TODOs:
- [ ] - Select a regression model, and start predicting.
- [ ] - If we are done early with that we can try to optimize the time spark / hadoop - mapreduce tasks take.
        - [ ] - repartition the data to use right amount cores. (24 cores)